# Tarea de fin de módulo
# Alejandro Martín del Campo

# Instrucciones

Tomando el siguiente archivo, realice una limpieza al mismo con la finalidad de obtener datos para ser analizados en un futuro. Cada persona es responsable de decidir cual columna desea modificar y hacer la sustitución de un valor por otro.  Lo importante es justificar esa acción. 
Realice 6 operaciones distintas de limpieza de datos y al final guarde su tratamiento en una base de datos SQL (Postgres, Mysql, Oracle) y en un archivo de tipo csv o json. Esta tarea debe usar la metodologia ETL y ELT.

## Entregable

Al final comparte un archivo con el paso a paso de la herramienta que usó, este archivo se debe colocar en la carpeta compartida que se adjunta

- Suba el archivo que compartio en la carpeta compartida

## Importo las librerías necesarias (pandas) y mi dataset, también veo que tipos de valores tengo

In [2]:
import pandas as pd
df = pd.read_csv('./movies.csv')
print('Información del dataset\n')
df.info()

Información del dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MOVIES    9999 non-null   object 
 1   YEAR      9355 non-null   object 
 2   GENRE     9919 non-null   object 
 3   RATING    8179 non-null   float64
 4   ONE-LINE  9999 non-null   object 
 5   STARS     9999 non-null   object 
 6   VOTES     8179 non-null   object 
 7   RunTime   7041 non-null   float64
 8   Gross     460 non-null    object 
dtypes: float64(2), object(7)
memory usage: 703.2+ KB


## Veo un poco los datos del dataset

In [3]:
df.head(10)

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"\nAction, Horror, Thriller",6.1,\nA woman with a mysterious illness is forced ...,\n Director:\nPeter Thorwarth\n| \n Star...,"21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\nAnimation, Action, Adventure",5.0,\nThe war for Eternia begins again in what may...,"\n \n Stars:\nChris Wood, \nSara...","17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"\nDrama, Horror, Thriller",8.2,\nSheriff Deputy Rick Grimes wakes up from a c...,"\n \n Stars:\nAndrew Lincoln, \n...","885,805",44.0,NaN
3,Rick and Morty,(2013– ),"\nAnimation, Adventure, Comedy",9.2,\nAn animated series that follows the exploits...,"\n \n Stars:\nJustin Roiland, \n...","414,849",23.0,NaN
4,Army of Thieves,(2021),"\nAction, Crime, Horror",NaN,"\nA prequel, set before the events of Army of ...",\n Director:\nMatthias Schweighöfer\n| \n ...,NaN,NaN,NaN
5,Outer Banks,(2020– ),"\nAction, Crime, Drama",7.6,\nA group of teenagers from the wrong side of ...,"\n \n Stars:\nChase Stokes, \nMa...","25,858",50.0,NaN
6,The Last Letter from Your Lover,(2021),"\nDrama, Romance",6.8,\nA pair of interwoven stories set in the past...,\n Director:\nAugustine Frizzell\n| \n S...,"5,283",110.0,NaN
7,Dexter,(2006–2013),"\nCrime, Drama, Mystery",8.6,"\nBy day, mild-mannered Dexter is a blood-spat...","\n \n Stars:\nMichael C. Hall, \...","665,387",53.0,NaN
8,Never Have I Ever,(2020– ),\nComedy,7.9,\nThe complicated life of a modern-day first g...,\n \n Stars:\nMaitreyi Ramakrish...,"34,530",30.0,NaN
9,Virgin River,(2019– ),"\nDrama, Romance",7.4,"\nSeeking a fresh start, nurse practitioner Me...",\n \n Stars:\nAlexandra Breckenr...,"27,279",44.0,NaN


## Limpieza

### 1. Eliminar los catacteres no deseados de la columna GENRE (saltos de linea)

In [4]:
df['GENRE'] = df['GENRE'].str.replace('\n', '')

### 2. Eliminar los catacteres no deseados de la columna Year y dejar solo el primer año que se encuentra

In [5]:
import re

# Convierte los valores de la columna "YEAR" a cadenas de texto (str)
df['YEAR'] = df['YEAR'].astype(str)
# Función para extraer el primer año de la columna "YEAR"
def extract_year(year_str):
    match = re.search(r'\d{4}', year_str)
    if match:
        return int(match.group())
    return None

# Aplica la función a la columna "YEAR"
df['YEAR'] = df['YEAR'].apply(lambda x: extract_year(x))
df['YEAR'] = pd.to_numeric(df['YEAR'], errors='coerce')

### 3. Limpiar la columna de VOTES y convertirla a valores numéricos

In [6]:
# remove all the "," in the votes colums in the df
df['VOTES'] = df['VOTES'].str.replace(',', '')
# change the Dtype of the VOTES column to float
df['VOTES'] = df['VOTES'].astype(float)

# Valicación
value_types = df['VOTES'].apply(lambda x: type(x).__name__).reset_index(name='Value_Type')
df['VOTES_TYPE'] = value_types['Value_Type']
value_counts = value_types.groupby('Value_Type').size().reset_index(name='Count')
print(value_counts)


  Value_Type  Count
0      float   9999


### 4. Ajustar los nombres de las columnas para que sean mas descriptivos y tengan la misma estructura

In [7]:
df.rename(columns={'MOVIES': 'Title', 'YEAR': 'Year', 'GENRE': 'Genre', 'RATING': 'Rating', 'ONE-LINE': 'Plot', 'STARS' : 'Cast' , 'VOTES': 'Votes', 'RunTime': 'Runtime'}, inplace=True)

### 5. Eliminar el salto de linea del principio de la columna Plot

In [8]:
df['Plot'] = df['Plot'].str.lstrip('\n')

### 6. Convertir la columna Gross a numérica

In [12]:
# Elimina el símbolo '$' de la columna 'Gross' y convierte a numérico
df['Gross'] = df['Gross'].str.replace('$', '')

# Convierte los valores que contienen 'M' al valor real de millones
df['Gross'] = df['Gross'].apply(lambda x: float(x.replace('M', '')) * 1e6 if isinstance(x, str) and 'M' in x else float(x))


# Cargo mi df a mi base de datos PostgreSQL

In [ ]:
import psycopg2
from sqlalchemy import create_engine

# Detalles de la conexión a tu base de datos PostgreSQL
db_config = {
    'dbname': nombre_base,
    'user': db_user,
    'password': db_pass,
    'host': 'localhost',  # Cambia esto a la dirección de tu servidor PostgreSQL si es necesario
    'port': '5432',       # Cambia esto al puerto de tu servidor PostgreSQL si es necesario
}

# Crea una conexión a la base de datos
connection = psycopg2.connect(**db_config)

# Crea un motor SQLAlchemy para usar con pandas
engine = create_engine(f'postgresql://{db_config["user"]}:{db_config["password"]}@{db_config["host"]}:{db_config["port"]}/{db_config["dbname"]}')

table_name = 'movies'

# Carga el DataFrame en la tabla PostgreSQL
df.to_sql(table_name, engine, if_exists='replace', index=False)

# Cierra la conexión
connection.close()